In [2]:
import os
import sys
import time
import pickle
from collections import defaultdict
from functools import partial
import torch

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FormatStrFormatter

# from safe_control_gym.envs.benchmark_env import Task
# from safe_control_gym.experiments.base_experiment import BaseExperiment
# from safe_control_gym.utils.configuration import ConfigFactory
# from safe_control_gym.utils.registration import make


from safe_control_gym.experiments.ROA_cartpole.utilities import *
from lyapnov import LyapunovNN, Lyapunov, QuadraticFunction, GridWorld_pendulum
from utilities import balanced_class_weights, dlqr, \
                      get_discrete_linear_system_matrices, onestep_dynamics

np.set_printoptions(threshold=sys.maxsize) # np print full array

class Options(object):
    def __init__(self, **kwargs):
        super(Options, self).__init__()
        self.__dict__.update(kwargs)

OPTIONS = Options(np_dtype              = np.float32,
                  torch_dtype           = torch.float32,
                  eps                   = 1e-8,                            # numerical tolerance
                  saturate              = True,                            # apply saturation constraints to the control input
                  use_zero_threshold    = True,                            # assume the discretization is infinitely fine (i.e., tau = 0)
                  pre_train             = True,                            # pre-train the neural network to match a given candidate in a supervised approach
                  dpi                   = 150,
                  num_cores             = 4,
                  num_sockets           = 1,
                #   tf_checkpoint_path    = "./tmp/lyapunov_function_learning.ckpt"
                )

# detect torch device
myDevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myDevice = torch.device("cpu")

/home/tumeilsy-mo-10/anaconda3/envs/safe/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
# Constants
dt = 0.01   # sampling time
g = 9.81    # gravity

# True system parameters
m = 0.15    # pendulum mass
L = 0.5     # pole length
b = 0.1     # rotational friction

# State and action normalizers
theta_max = np.deg2rad(180)                     # angular position [rad]
omega_max = np.deg2rad(360)                     # angular velocity [rad/s]
# u_max     = g * m * L * np.sin(np.deg2rad(60))  # torque [N.m], control action
u_max = 0.5

state_norm = (theta_max, omega_max)
action_norm = (u_max,)

# Dimensions and domains
state_dim     = 2
action_dim    = 1
state_limits  = np.array([[-1., 1.]] * state_dim)
action_limits = np.array([[-1., 1.]] * action_dim)

In [4]:
# Initialize system class and its linearization
pendulum = InvertedPendulum(m, L, b, dt, [state_norm, action_norm])
A, B = pendulum.linearize()
# print("A\n ", A)
# print("B\n ", B)
# dynamics = pendulum.__call__
dynamics = pendulum.__call__

# # test dynamics with state [0.5, 0] and action [0.5]
# x = torch.tensor([0.5, 0.], dtype=OPTIONS.torch_dtype)
# u = torch.tensor([-0.5], dtype=OPTIONS.torch_dtype)
# x = np.array([0.5, 0.])
# u = np.array([-0.5])
# print("x: ", x)
# print("u: ", u)
# state_action = np.concatenate([x, u])
# state_action = torch.cat([x, u])
# print("state_action: ", state_action)

# x_next = dynamics(state_action)
# print("x_next: ", x_next)
# assert x_next.shape == x.shape
# print("Dynamics shape test passed!")

In [5]:
state_constraints = np.array([[-theta_max, theta_max], [-omega_max, omega_max]])

print('state_constraints: ', state_constraints)
num_states = 100

grid_limits = np.array([[-1., 1.], ] * state_dim)
# state_discretization = gridding(state_dim, state_constraints=None, num_states = 100)
state_discretization = GridWorld_pendulum(grid_limits, num_states)
# state_discretization = gridding(state_dim, state_constraints, num_states = 100)
print('state_discretization.all_points.shape: ', state_discretization.all_points.shape)

state_constraints:  [[-3.14159265  3.14159265]
 [-6.28318531  6.28318531]]
state_discretization.all_points.shape:  (10000, 2)


In [6]:
# Discretization constant
if OPTIONS.use_zero_threshold:
    tau = 0.0
else:
    tau = np.sum(state_discretization.unit_maxes) / 2

print('Grid size: {}'.format(state_discretization.nindex))
print('Discretization constant (tau): {}'.format(tau))

# Set initial safe set as a ball around the origin (in normalized coordinates)
cutoff_radius    = 0.1
initial_safe_set = np.linalg.norm(state_discretization.all_points, ord=2, axis=1) <= cutoff_radius
print('state_discretization.all_points.shape: ', state_discretization.all_points.shape)
print('initial_safe_set.sum(): ', initial_safe_set.shape)

Grid size: 10000
Discretization constant (tau): 0.0
state_discretization.all_points.shape:  (10000, 2)
initial_safe_set.sum():  (10000,)


In [10]:
Q = np.identity(state_dim).astype(OPTIONS.np_dtype)     # state cost matrix
Q = np.diag([5, 1])
R = 1* np.identity(action_dim).astype(OPTIONS.np_dtype)    # action cost matrix
# K, P_lqr = safe_learning.utilities.dlqr(A, B, Q, R)
K, P_lqr = dlqr(A, B, Q, R) 
# print('K',K)

policy = lambda x: -K @ x
if OPTIONS.saturate:
    # policy = lambda x: np.clip(-K @ x, -u_max, u_max)
    policy = lambda x: np.clip(-K @ x, -1, 1)

cl_dynamics = lambda x: dynamics(np.concatenate([x, policy(x)]))

# test the closed-loop dynamics with state [0.5, 0]
x = np.array([0.20, 0.])
for i in range(100):
    x = cl_dynamics(x)
    print("x: ", x)
    

x:  [ 0.19997438 -0.00283275]
x:  [ 0.19989275 -0.00559486]
x:  [ 0.19975647 -0.00829254]
x:  [ 0.19956678 -0.01093174]
x:  [ 0.19932478 -0.01351815]
x:  [ 0.19903149 -0.01605725]
x:  [ 0.1986878  -0.01855429]
x:  [ 0.19829452 -0.02101433]
x:  [ 0.19785233 -0.02344227]
x:  [ 0.19736183 -0.0258428 ]
x:  [ 0.19682354 -0.02822047]
x:  [ 0.19623787 -0.03057971]
x:  [ 0.19560515 -0.03292479]
x:  [ 0.19492562 -0.03525986]
x:  [ 0.19419945 -0.03758898]
x:  [ 0.19342672 -0.03991608]
x:  [ 0.19260745 -0.04224503]
x:  [ 0.19174154 -0.04457961]
x:  [ 0.19082887 -0.04692351]
x:  [ 0.18986921 -0.04928039]
x:  [ 0.18886227 -0.05165383]
x:  [ 0.18780769 -0.05404738]
x:  [ 0.18670502 -0.05646454]
x:  [ 0.18555378 -0.05890879]
x:  [ 0.18435338 -0.06138358]
x:  [ 0.18310319 -0.06389236]
x:  [ 0.18180249 -0.06643856]
x:  [ 0.1804505  -0.06902561]
x:  [ 0.17904638 -0.07165695]
x:  [ 0.17758921 -0.07433604]
x:  [ 0.176078   -0.07706636]
x:  [ 0.1745117 -0.0798514]
x:  [ 0.17288918 -0.08269471]
x:  [ 0.1712

In [7]:
L_pol = lambda x: np.linalg.norm(-K, 1)

L_dyn = lambda x: np.linalg.norm(A, 1) + np.linalg.norm(B, 1) * L_pol(x)

In [8]:
lyapunov_function = QuadraticFunction(P_lqr)

# Approximate local Lipschitz constants with gradients
# grad_lyapunov_function = lambda x: 2 * P_lqr @ x
grad_lyapunov_function = lambda x: 2 * torch.tensor(P_lqr, dtype=torch.float32) @ x
# L_v = lambda x: tf.norm(grad_lyapunov_function(x), ord=1, axis=1, keepdims=True)
L_v = lambda x: torch.norm(grad_lyapunov_function(x), p=1, dim=-1, keepdim=True)

# Initialize Lyapunov class
lyapunov_lqr = Lyapunov(state_discretization, lyapunov_function, cl_dynamics, L_dyn, L_v, tau, policy, initial_safe_set)
# print the first 10 states in the safe set
lyapunov_lqr.discretization.all_points[1:10]
# print('lyaupunov_lqr.discretization.all_points[1:10]\n', lyapunov_lqr.discretization.all_points[1:10])
lyapunov_lqr.update_values()
lyapunov_lqr.update_safe_set()
# print('lyapunov_lqr.safe_set\n', lyapunov_lqr.safe_set[1:10])
# print('lyapunov_lqr.values\n', lyapunov_lqr.values[1:10])
# print('lyapunov_lqr.c_max\n', lyapunov_lqr.c_max)
print('lyapunov_lqr.safe_set.sum()\n', lyapunov_lqr.safe_set.sum())

lyapunov_lqr.safe_set.sum()
 2006


In [9]:
horizon = 500
tol = 0.1
# roa, trajectories = compute_roa_pendulum(lyapunov_lqr.discretization, cl_dynamics, horizon, tol, no_traj=False)
compute_new_roa = False
# compute_new_roa = True

# get the current path of this jupyter notebook
script_dir = os.getcwd()
roa_file_name = 'roa_pendulum.npy'
traj_file_name = 'traj_pendulum.npy'
# append the file name to the current path
roa_file_name = os.path.join(script_dir, roa_file_name)
traj_file_name = os.path.join(script_dir, traj_file_name)
if not compute_new_roa:
    # load the pre-saved ROA to avoid re-computation
    roa = np.load(roa_file_name)
    trajectories = np.load(traj_file_name)
else:
    roa, trajectories = compute_roa_pendulum(lyapunov_lqr.discretization, cl_dynamics, horizon, tol, no_traj=False)
    np.save(roa_file_name, roa)
    np.save(traj_file_name, trajectories)
    # exit()

print('True ROA size:{}\n'.format(int(roa.sum())))
print('')

True ROA size:2518




In [16]:
######################## define Lyapunov NN ########################
# initialize Lyapunov NN
from lyapnov import LyapunovNN

layer_dim = [64, 64, 64]
# layer_dim = [128, 128, 128]
activations = [torch.nn.Tanh(), torch.nn.Tanh(), torch.nn.Tanh()]
nn = LyapunovNN(state_dim, layer_dim, activations)
print('nn: ', nn)

for name, param in nn.named_parameters():
    if param.requires_grad:
        print(name, param.data)

# forward some random states
x = torch.randn(state_dim,)
# print('x: ', x)
# print('nn(x): ', nn(x))

for name, param in nn.named_parameters():
    if param.requires_grad:
        print(name, param.data)


nn:  LyapunovNN(
  (layers): ModuleList()
)


In [ ]:
# approximate local Lipschitz constant with gradient
grad_lyapunov_function = \
    lambda x: torch.autograd.grad(nn(x), x, \
                    torch.ones_like(nn(x)), allow_unused=True,)[0]
lyapunov_nn = Lyapunov(state_discretization, nn, \
                          cl_dynamics, L_dyn, L_v, tau, policy, \
                          initial_safe_set)
lyapunov_nn.update_values()
lyapunov_nn.update_safe_set()

In [ ]:
#########################################################################
# train the parameteric Lyapunov candidate in order to expand the verifiable
# safe set toward the brute-force safe set
test_classfier_loss = []
test_decrease_loss   = []
roa_estimate         = np.copy(lyapunov_nn.safe_set)

# grid              = lyapunov_lqr.discretization
grid              = lyapunov_nn.discretization
c_max             = [lyapunov_nn.c_max, ]
safe_set_fraction = [lyapunov_nn.safe_set.sum() / grid.nindex, ]
print('safe_set_fraction', safe_set_fraction)

In [ ]:
######################### traning hyperparameters #######################
outer_iters = 5
inner_iters = 10
horizon     = 100
test_size   = int(1e4)

safe_level = 1
lagrange_multiplier = 5000
level_multiplier = 1.3
learning_rate = 5e-3
batch_size    = int(1e3)

optimizer = torch.optim.SGD(lyapunov_nn.lyapunov_function.parameters(), lr=learning_rate)


In [ ]:
############################# training loop #############################
print('Current metrics ...')
c = lyapunov_nn.c_max
num_safe = lyapunov_nn.safe_set.sum()
print('Safe level (c_k): {}'.format(c))
print('Safe set size: {} ({:.2f}% of grid, \
        {:.2f}% of ROA)\n'.format(int(num_safe), \
        100 * num_safe / grid.nindex, 100 * num_safe / roa.sum()))
print('')
time.sleep(0.5)
for _ in range(outer_iters):
    print('Iteration (k): {}'.format(len(c_max)))
    time.sleep(0.5)

    ## Identify the "gap" states, i.e., those between V(c_k) 
    ## and V(a * c_k) for a > 1
    c = lyapunov_nn.c_max
    idx_small = lyapunov_nn.values.ravel() <= c
    # print('lyapunov_nn.values.ravel()', lyapunov_nn.values.ravel())
    # print('c', c)
    # print('level_multiplier', level_multiplier)
    idx_big   = lyapunov_nn.values.ravel() <= level_multiplier * c
    # print('idx_small', idx_small)
    # print('idx_big', idx_big)
    idx_gap   = np.logical_and(idx_big, ~idx_small)

    ## Forward-simulate "gap" states to determine 
    ## which ones we can add to our ROA estimate
    gap_states = grid.all_points[idx_gap]
    print('gap_states\n', gap_states)
    print('gap_states.shape', gap_states.shape)
    # input('press enter to continue')
    gap_future_values = np.zeros((gap_states.shape[0], 1))
    for state_idx in range(gap_states.shape[0]):
        # !! when using dynamics, the state can go out of the bound
        for _ in range(horizon):
            # print('gap_states[state_idx]', gap_states[state_idx])
            # print('gap_states[state_idx].shape', gap_states[state_idx].shape)
            # # print('policy(gap_states[state_idx])', policy(gap_states[state_idx]))
            # print('dynamics(gap_states[state_idx])', \
            #         dynamics(gap_states[state_idx]))
            # print('dynamics(gap_states[state_idx]).shape', \
            #         dynamics(gap_states[state_idx]).shape)
            # dynamics return the next state in the form of (4, 1)
            # to feed into gap_states[state_idx], we need to reshape it to (4,)
            gap_states[state_idx] = np.reshape(cl_dynamics(gap_states[state_idx]), -1)
        # use the safe-control-gym to simulate the trajectory
        # init_state = gap_states[state_idx]
        # init_state_dict = {'init_x': init_state[0], 'init_x_dot': init_state[1], \
                            # 'init_theta': init_state[2], 'init_theta_dot': init_state[3]}
        # init_state, _ = random_env.reset(init_state = init_state_dict)
        # print('init_state', init_state)
        # static_env = env_func(gui=False, random_state=False, init_state=init_state)
        # static_train_env = env_func(gui=False, randomized_init=False, init_state=init_state)
        # Create experiment, train, and run evaluation
        # experiment = BaseExperiment(env=static_env, ctrl=ctrl, train_env=static_train_env)
        # simulate the gap state in safe-control-gym for only pre-defined horizon
        # trajs_data, _ = experiment.run_evaluation(training=True, n_steps=horizon, verbose=False)
        # print('trajectory data\n', trajs_data)
        # print('trajs_data\n', trajs_data['info'][0][-1])
        # print('obs[0]\n', trajs_data['obs'][0][-1])
        # gap_states[state_idx] = trajs_data['obs'][0][-1]
        # print('gap_states[state_idx]', gap_states[state_idx])
        # print('gap_states[state_idx].shape', gap_states[state_idx].shape)
        # print('gap_states[state_idx] type', type(gap_states[state_idx]))
        gap_future_values[state_idx] = (lyapunov_nn.lyapunov_function(\
                                    gap_states[state_idx])).detach().numpy()
        # exit()
        # Close environments
        # static_env.close()
        # static_train_env.close()
    # print('gap_states\n', gap_states)
    # print('gap_future_values\n', gap_future_values)
    # reshape c to match the dim of gap_future_values
    c_vec = np.ones_like(gap_future_values) * c
    # concatenate all gap states with gap future values as a sanity check
    # also concatenate current safe set with current safe level
    res = np.hstack((c_vec, gap_future_values, gap_states))
    print('gap state and future values res\n', res)
    print('\n')
    # print('roa_estimate[idx_gap] before ior', roa_estimate[idx_gap])
    # print('roa_estimate[idx_gap] shape', roa_estimate[idx_gap].shape)
    roa_estimate[idx_gap] |= (gap_future_values <= c).ravel()
    # print('roa_estimate[idx_gap] after ior', roa_estimate[idx_gap])
    print('roa_estimate[idx_gap] shape', roa_estimate[idx_gap].shape)
    # input('press enter to continue')

    ## Identify the class labels for our current ROA estimate 
    ## and the expanded level set
    target_idx = np.logical_or(idx_big, roa_estimate)
    target_set = grid.all_points[target_idx]
    target_labels = roa_estimate[target_idx]\
                    .astype(OPTIONS.np_dtype).reshape([-1, 1])
    print('target_labels\n', target_labels.T)
    idx_range = target_set.shape[0]
    # print('idx_range', idx_range)
    # exit()

    ## test set
    # idx_batch = tf.random_uniform([batch_size, ], 0, idx_range, dtype=tf.int32, name='batch_sample')
    idx_test = np.random.randint(0, idx_range, size=(test_size, ))
    test_set = target_set[idx_test]
    test_labels = target_labels[idx_test]

    # stochastic gradient descent for classification
    for _ in range(inner_iters):
        lyapunov_nn.lyapunov_function.train()
        # training step
        # safe_level = lyapunov_nn.c_max
        idx_batch_eval = np.random.randint(0, idx_range, size=(batch_size, ))
        training_states = target_set[idx_batch_eval]
        num_training_states = training_states.shape[0]
        print('training_states\n', training_states)
        
        # True class labels, converted from Boolean ROA labels {0, 1} to {-1, 1}
        roa_labels = target_labels[idx_batch_eval]
        class_label = 2 * roa_labels - 1
        class_label = torch.tensor(class_label, dtype=torch.float32, device=myDevice)
        print('roa_labels\n', roa_labels.T)

        # concatenate all training states with class labels as a sanity check
        res = np.hstack((class_label, training_states))
        print('training states and class labels res\n', res)
        print('\n')
        # exit()
        # Signed, possibly normalized distance from the decision boundary
        # print('safe_level\n', safe_level)
        decision_distance_for_states = torch.zeros((num_training_states, 1), dtype=torch.float32, device=myDevice)                                                   
        for state_idx in range(num_training_states):
            # decision_distance_for_states[state_idx] = safe_level - lyapunov_nn.lyapunov_function(training_states[state_idx].reshape(-1, 1))
            decision_distance_for_states[state_idx] = lyapunov_nn.lyapunov_function(training_states[state_idx])
        
        decision_distance = safe_level - decision_distance_for_states
        print('decision_distance\n', decision_distance.T)
        # exit()
        
        # Perceptron loss with class weights
        class_weights, class_counts = balanced_class_weights(roa_labels.astype(bool))
        # print('class_weights\n', class_weights.T)
        # classifier_loss = class_weights * tf.maximum(- class_labels * decision_distance, 0, name='classifier_loss')
        # print('class_weights\n', class_weights)
        # print('class_weights type\n', type(class_weights))
        # convert class_weights to torch tensor
        class_weights = torch.tensor(class_weights, dtype=torch.float32, device=myDevice)
        classifier_loss = class_weights * torch.max(- class_label * decision_distance, torch.zeros_like(decision_distance, device=myDevice)) 
        # print('classifier_loss\n', classifier_loss.T)
        # input('press enter to continue')
        # exit()
        # Enforce decrease constraint with Lagrangian relaxation
        # decrease_loss = roa_labels * tf.maximum(tf_dv_nn, 0) / tf.stop_gradient(tf_values_nn + OPTIONS.eps)
        torch_dv_nn = torch.zeros((num_training_states, 1), dtype=torch.float32, device=myDevice)
        for state_idx in range(num_training_states):
            future_state = np.reshape(cl_dynamics(training_states[state_idx]), -1)
            torch_dv_nn[state_idx] = lyapunov_nn.lyapunov_function(\
                                future_state) - \
                                lyapunov_nn.lyapunov_function(training_states[state_idx])
        print('torch_dv_nn\n', torch_dv_nn.T)
        # exit()
        roa_labels = torch.tensor(roa_labels, dtype=torch.float32, device=myDevice).detach()
        training_states_forwards = torch.zeros((num_training_states, 1), dtype=torch.float32, device=myDevice).detach()
        for state_idx in range(num_training_states):
            training_states_forwards[state_idx] = lyapunov_nn.lyapunov_function(training_states[state_idx])
        # TODO: check ROA labels
        print('torch.max(torch_dv_nn, torch.zeros_like(torch_dv_nn))\n', torch.max(torch_dv_nn, torch.zeros_like(torch_dv_nn)).T)
        decrease_loss = roa_labels * torch.max(torch_dv_nn, torch.zeros_like(torch_dv_nn))  \
                            # /(training_states_forwards + OPTIONS.eps)
        print('decrease_loss\n', decrease_loss.T)
        loss = torch.mean(classifier_loss + lagrange_multiplier * decrease_loss)
        print('loss\n', loss)
        # input('press enter to continue')
        optimizer.zero_grad() # zero gradiants for every batch !!
        loss.backward()
        # print('optimizer\n', optimizer)
        # exit()
        optimizer.step()
        # exit()
    
    ## Update Lyapunov values and ROA estimate, 
    ## based on new parameter values
    print('lyapunov_nn.values before update\n', lyapunov_nn.values[1:10])
    lyapunov_nn.update_values()  
    print('lyapunov_nn.values\n', lyapunov_nn.values[1:10])
    lyapunov_nn.update_safe_set()
    roa_estimate |= lyapunov_nn.safe_set

    c_max.append(lyapunov_nn.c_max)
    safe_set_fraction.append(lyapunov_nn.safe_set.sum() / grid.nindex)
    print('Current safe level (c_k): {}'.format(c_max[-1]))
    print('Safe set size: {} ({:.2f}% of grid, {:.2f}% of ROA)\n'.format(
                            int(lyapunov_nn.safe_set.sum()), \
                            100 * safe_set_fraction[-1], \
                            100 * safe_set_fraction[-1] * roa.size / roa.sum()\
                                ))
    # input('press enter to continue')

print('c_max', c_max)
print('safe_set_fraction', safe_set_fraction)

In [ ]:
################################ plotting ################################
fig = plt.figure(figsize=(8, 3), dpi=OPTIONS.dpi, frameon=False)
fig.subplots_adjust(wspace=0.35)
plot_limits = np.column_stack((- np.rad2deg([theta_max, omega_max]), np.rad2deg([theta_max, omega_max])))

# ax = plt.subplot(121)
ax = plt.subplot(111)
alpha = 1
colors = [None] * 4
colors[0] = (0, 158/255, 115/255)       # ROA - bluish-green
colors[1] = (230/255, 159/255, 0)       # NN  - orange
colors[2] = (0, 114/255, 178/255)       # LQR - blue
colors[3] = (240/255, 228/255, 66/255)  # SOS - yellow

# True ROA
z = roa.reshape(grid.num_points)
# print(z.shape)
# print(z)
ax.contour(z.T, origin='lower', extent=plot_limits.ravel(), colors=(colors[0],), linewidths=1)
ax.imshow(z.T, origin='lower', extent=plot_limits.ravel(), cmap=binary_cmap(colors[0]), alpha=alpha)

# # Neural network
z = lyapunov_nn.safe_set.reshape(grid.num_points)
ax.contour(z.T, origin='lower', extent=plot_limits.ravel(), colors=(colors[1],), linewidths=1)
ax.imshow(z.T, origin='lower', extent=plot_limits.ravel(), cmap=binary_cmap(colors[1]), alpha=alpha)

# LQR
z = lyapunov_lqr.safe_set.reshape(grid.num_points)
ax.contour(z.T, origin='lower', extent=plot_limits.ravel(), colors=(colors[2],), linewidths=1)
ax.imshow(z.T, origin='lower', extent=plot_limits.ravel(), cmap=binary_cmap(colors[2]), alpha=alpha)



# Plot some trajectories
N_traj = 11
skip = int(grid.num_points[0] / N_traj)
sub_idx = np.arange(grid.nindex).reshape(grid.num_points)
sub_idx = sub_idx[::skip, ::skip].ravel()
sub_trajectories = trajectories[sub_idx, :, :]
sub_states = grid.all_points[sub_idx]
for n in range(sub_trajectories.shape[0]):
    x = sub_trajectories[n, 0, :] * np.rad2deg(theta_max)
    y = sub_trajectories[n, 1, :] * np.rad2deg(omega_max)
    ax.plot(x, y, 'k--', linewidth=0.25)
sub_states = grid.all_points[sub_idx]
dx_dt = np.zeros_like(sub_states)
for state_idx in range(sub_states.shape[0]):
    dx_dt[state_idx, :] = (cl_dynamics(sub_states[state_idx, :]) - sub_states[state_idx, :])/dt

# dx_dt = (tf_future_states.eval({tf_states: sub_states}) - sub_states) / dt
dx_dt = dx_dt / np.linalg.norm(dx_dt, ord=2, axis=1, keepdims=True)
ax.quiver(sub_states[:, 0] * np.rad2deg(theta_max), sub_states[:, 1] * np.rad2deg(omega_max), dx_dt[:, 0], dx_dt[:, 1], 
          scale=None, pivot='mid', headwidth=3, headlength=6, color='k')

ax.set_title('ROA of pendulum under an LQR prior policy (l={:.2f})'.format(L))
ax.set_aspect(theta_max / omega_max / 1.2)
ax.set_xlim(plot_limits[0])
ax.set_ylim(plot_limits[1])
ax.set_xlabel(r'angle [deg]')
ax.set_ylabel(r'angular velocity [deg/s]')
ax.xaxis.set_ticks(np.arange(-180, 181, 60))
ax.yaxis.set_ticks(np.arange(-360, 361, 120))

proxy = [plt.Rectangle((0,0), 1, 1, fc=c) for c in colors]    
legend = ax.legend(proxy, [r'Brute-forced ROA'], loc='upper right')
legend.get_frame().set_alpha(1.)



plt.show()